# Basic Examples with Different Protocols

## Prerequisites

 * A kubernetes cluster with kubectl configured
 * curl
 * grpcurl
 * pygmentize
 

## Setup Seldon Core

Use the setup notebook to [Setup Cluster](https://docs.seldon.io/projects/seldon-core/en/latest/examples/seldon_core_setup.html) to setup Seldon Core with an ingress - either Ambassador or Istio.

Then port-forward to that ingress on localhost:8003 in a separate terminal either with:

 * Ambassador: `kubectl port-forward $(kubectl get pods -n seldon -l app.kubernetes.io/name=ambassador -o jsonpath='{.items[0].metadata.name}') -n seldon 8003:8080`
 * Istio: `kubectl port-forward $(kubectl get pods -l istio=ingressgateway -n istio-system -o jsonpath='{.items[0].metadata.name}') -n istio-system 8003:80`

## Install Seldon Analytics

In [1]:
!helm install seldon-core-analytics ../../../helm-charts/seldon-core-analytics -n seldon-system --wait

Error: cannot re-use a name that is still in use


In [2]:
!kubectl create namespace seldon

Error from server (AlreadyExists): namespaces "seldon" already exists


In [3]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

Context "kind-kind" modified.


In [4]:
import json
import time

## Custom Metrics

In [5]:
!pygmentize model.yaml

apiVersion: machinelearning.seldon.io/v1
kind: SeldonDeployment
metadata:
  name: seldon-model
spec:
  name: test-deployment
  predictors:
  - componentSpecs:
    - spec:
        containers:
        - image: seldonio/model-with-metrics:0.1
          name: classifier
    graph:
      children: []
      name: classifier
      type: MODEL
    name: example
    replicas: 1


In [6]:
!kubectl create -f model.yaml

seldondeployment.machinelearning.seldon.io/seldon-model created


In [7]:
!kubectl rollout status deploy/$(kubectl get deploy -l seldon-deployment-id=seldon-model \
                                 -o jsonpath='{.items[0].metadata.name}')

Waiting for deployment "seldon-model-example-0-classifier" rollout to finish: 0 of 1 updated replicas are available...
deployment "seldon-model-example-0-classifier" successfully rolled out


In [8]:
responseRaw=!curl -s -d '{"data": {"ndarray":[[1.0, 2.0, 5.0]]}}'    -X POST http://localhost:8003/seldon/seldon/seldon-model/api/v1.0/predictions    -H "Content-Type: application/json"

In [9]:
response=json.loads(responseRaw[0])
print(response)
assert(len(response["meta"]["metrics"]) == 3)

{'data': {'names': ['t:0', 't:1', 't:2'], 'ndarray': [[1.0, 2.0, 5.0]]}, 'meta': {'metrics': [{'key': 'mycounter', 'type': 'COUNTER', 'value': 1}, {'key': 'mygauge', 'type': 'GAUGE', 'value': 100}, {'key': 'mytimer', 'type': 'TIMER', 'value': 20.2}]}}


In [10]:
print("Waiting so metrics can be scraped")
time.sleep(10)

Waiting so metrics can be scraped


In [11]:
%%writefile get-metrics.sh

kubectl run --quiet=true -it --rm curlmetrics --image=tutum/curl --restart=Never -- \
    curl -s seldon-core-analytics-prometheus-seldon.seldon-system/api/v1/query?query=mycounter_total

Overwriting get-metrics.sh


In [12]:
responseRaw =! bash get-metrics.sh

In [13]:
responseRaw[0]

'{"status":"success","data":{"resultType":"vector","result":[{"metric":{"__name__":"mycounter_total","app":"seldon-model-example-0-classifier","app_kubernetes_io_managed_by":"seldon-core","deployment_name":"seldon-model","fluentd":"true","image_name":"seldonio/model-with-metrics","image_version":"0.1","instance":"10.244.1.41:6000","job":"kubernetes-pods","kubernetes_namespace":"seldon","kubernetes_pod_name":"seldon-model-example-0-classifier-6d4897ccf4-d8nhw","method":"predict","model_image":"seldonio/model-with-metrics","model_name":"classifier","model_version":"0.1","pod_template_hash":"6d4897ccf4","predictor_name":"example","predictor_version":"example","seldon_app":"seldon-model-example","seldon_app_svc":"seldon-model-example-classifier","seldon_deployment_id":"seldon-model","seldon_deployment_name":"seldon-model","seldon_io_default":"true","seldon_io_model":"true","version":"example","worker_id":"42"},"value":[1604236241.248,"1"]}]}}'

In [14]:
response=json.loads(responseRaw[0])
print(response)
assert(response['data']["result"][0]["metric"]["__name__"]=='mycounter_total')

{'status': 'success', 'data': {'resultType': 'vector', 'result': [{'metric': {'__name__': 'mycounter_total', 'app': 'seldon-model-example-0-classifier', 'app_kubernetes_io_managed_by': 'seldon-core', 'deployment_name': 'seldon-model', 'fluentd': 'true', 'image_name': 'seldonio/model-with-metrics', 'image_version': '0.1', 'instance': '10.244.1.41:6000', 'job': 'kubernetes-pods', 'kubernetes_namespace': 'seldon', 'kubernetes_pod_name': 'seldon-model-example-0-classifier-6d4897ccf4-d8nhw', 'method': 'predict', 'model_image': 'seldonio/model-with-metrics', 'model_name': 'classifier', 'model_version': '0.1', 'pod_template_hash': '6d4897ccf4', 'predictor_name': 'example', 'predictor_version': 'example', 'seldon_app': 'seldon-model-example', 'seldon_app_svc': 'seldon-model-example-classifier', 'seldon_deployment_id': 'seldon-model', 'seldon_deployment_name': 'seldon-model', 'seldon_io_default': 'true', 'seldon_io_model': 'true', 'version': 'example', 'worker_id': '42'}, 'value': [1604236241.2

In [15]:
responseRaw=!cd ../../../executor/proto && grpcurl -d '{"data":{"ndarray":[[1.0,2.0,5.0]]}}' \
         -rpc-header seldon:seldon-model -rpc-header namespace:seldon \
         -plaintext \
         -proto ./prediction.proto  0.0.0.0:8003 seldon.protos.Seldon/Predict

In [16]:
response=json.loads("".join(responseRaw))
print(response)
assert(len(response["meta"]["metrics"]) == 3)

{'meta': {'metrics': [{'key': 'mycounter', 'value': 1}, {'key': 'mygauge', 'type': 'GAUGE', 'value': 100}, {'key': 'mytimer', 'type': 'TIMER', 'value': 20.2}]}, 'data': {'names': ['t:0', 't:1', 't:2'], 'ndarray': [[1, 2, 5]]}}


In [17]:
print("Waiting so metrics can be scraped")
time.sleep(10)

Waiting so metrics can be scraped


In [18]:
responseRaw =! bash get-metrics.sh

In [19]:
response=json.loads(responseRaw[0])
print(response)
assert(response['data']["result"][0]["metric"]["__name__"]=='mycounter_total')
assert(response['data']["result"][0]["metric"]["image_name"]=='seldonio/model-with-metrics')

{'status': 'success', 'data': {'resultType': 'vector', 'result': [{'metric': {'__name__': 'mycounter_total', 'app': 'seldon-model-example-0-classifier', 'app_kubernetes_io_managed_by': 'seldon-core', 'deployment_name': 'seldon-model', 'fluentd': 'true', 'image_name': 'seldonio/model-with-metrics', 'image_version': '0.1', 'instance': '10.244.1.41:6000', 'job': 'kubernetes-pods', 'kubernetes_namespace': 'seldon', 'kubernetes_pod_name': 'seldon-model-example-0-classifier-6d4897ccf4-d8nhw', 'method': 'predict', 'model_image': 'seldonio/model-with-metrics', 'model_name': 'classifier', 'model_version': '0.1', 'pod_template_hash': '6d4897ccf4', 'predictor_name': 'example', 'predictor_version': 'example', 'seldon_app': 'seldon-model-example', 'seldon_app_svc': 'seldon-model-example-classifier', 'seldon_deployment_id': 'seldon-model', 'seldon_deployment_name': 'seldon-model', 'seldon_io_default': 'true', 'seldon_io_model': 'true', 'version': 'example', 'worker_id': '42'}, 'value': [1604236276.6

In [20]:
!kubectl delete -f model.yaml

seldondeployment.machinelearning.seldon.io "seldon-model" deleted


In [21]:
!helm delete seldon-core-analytics --namespace seldon-system

release "seldon-core-analytics" uninstalled
